## Imports

In [ ]:
import time
from datetime import datetime
import logging
import pytz
import requests
from pymongo import MongoClient
import config
import pandas as pd

FORMAT = "%(levelname)s - %(asctime)s - %(message)s"
logging.basicConfig(
        filename='cit.log',
        filemode='w',
        format=FORMAT,
        level=logging.ERROR)
LOGGER = logging.getLogger()

#### Dev

In [ ]:
import pprint
from IPython.display import display

pp = pprint.PrettyPrinter(indent=4)